### Camada Silver: Limpeza e Transformação

Aplicar transformações e desnormalizar os dados na camada Silver. Use particionamento para melhorar o desempenho de leitura e escrita.

In [0]:
# Importar as bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Iniciar a SparkSession com configurações otimizadas
# Define um número fixo de partições para shuffle, melhorando o paralelismo                 
# Define o tamanho máximo de partições para evitar muitos arquivos pequenos        
# Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita    
# Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados
spark = SparkSession.builder \
    .appName("Transformação Data Silver") \
    .config("spark.sql.shuffle.partitions", "200")  \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Definir caminhos de armazenamento no Data Lake
# Ler dados na Bronze e Armazenar Silver

bronze_path = "/Volumes/workspace/store/bronze/vendas"
silver_path = "/Volumes/workspace/store/silver/vendas"

###Ler o dados da camada Bronze para transformação na camada Silver

In [0]:
# Ler dados da camada Bronze
df_bronze = spark.read.format("parquet").load(bronze_path)
display(df_bronze)

### Limpeza dos dados (Cleaning Data)
A limpeza de dados é um processo crucial para garantir a qualidade dos dados. Isso envolve a remoção de dados duplicados ou incorretos, a padronização de formatos e valores de dados e o enriquecimento de dados com informações adicionais. Além disso, é importante verificar e corrigir problemas de qualidade, como erros e inconsistências, para garantir que os dados sejam precisos e confiáveis.


In [0]:
from pyspark.sql.functions import (
    to_date, col, lower, expr, regexp_replace, split, concat, format_number
)

# Realizar transformações necessárias, incluindo a manipulação do campo EmailNome IdCampanha
df_silver = df_bronze \
    .withColumn("Data", to_date(col("Data"), "yyyy-MM-dd")) \
    .withColumn("Email", lower(expr("regexp_replace(split(EmailNome, ':')[0], '[()]', '')"))) \
    .withColumn("Nome", expr("split(split(EmailNome, ':')[1], ', ')")) \
    .withColumn("Nome", expr("concat(Nome[1], ' ', Nome[0])")) \
    .withColumn("Cidade", expr("split(Cidade, ',')[0]")) \
    .withColumn("TotalVendas", format_number(col("PrecoUnitario") * col("Unidades"),2)) \
    .withColumn("PrecoUnitario", format_number(col("PrecoUnitario"), 2)) \
    .withColumn("CustoUnitario", format_number(col("CustoUnitario"), 2)) \
    .drop("EmailNome") \
    .drop("IdCampanha")

display(df_silver)

### Gravar transformações Silver

Particionamento por ano e mês para otimizar consultas baseadas em data, com recomendação de tamanho de arquivo em formato Parquet

In [0]:
# Particionamento por ano e mês para otimizar consultas baseadas em data, com recomendação de tamanho de arquivo

df_silver.withColumn("Ano", year("Data")) \
         .withColumn("Mes", month("Data")) \
         .write.option("maxRecordsPerFile", 50000) \
         .partitionBy("Ano", "Mes") \
         .format("parquet") \
         .mode("overwrite") \
         .save(silver_path)

#Contagem de registros
df_silver.count()

%md
**Justificativa para particionamento:**

partitionBy("Ano", "Mes"): Particionar os dados pelas coluna Ano e Mês ajuda a otimizar a leitura quando queremos filtrar ou consultar dados baseados em periodos específicos. Isso reduz o número de arquivos escaneados em consultas, melhorando a performance.



### Limpando a Memória

In [0]:
import gc
gc.collect()

In [0]:
del df_bronze
del df_silver